# Siamese network implementation with the strips dataset

last hack

In [1]:
import os
import numpy as np
import random as rng
import cv2
import json

In [2]:
info_data = []
with open('/home/data/strips_socrates/dataset_info.json') as json_file: 
    info_data = json.load(json_file) 

In [3]:
STRIP_SIZE = 256
SUB_STRIP_SIZE = 128

In [4]:
def get_batch(info_data, batch_size, examples, with_id=False, dataset = 'train'):
    pairs = [np.zeros((batch_size, examples, STRIP_SIZE, STRIP_SIZE, 3)) for i in range(2)]
    labels = np.zeros((batch_size, ))
    labels[batch_size//2:] = 1
    split_index = int((len(info_data)-1) * 0.8)
    if dataset == 'train':
        students = [rng.randint(0, split_index) for _ in range(batch_size)] 
    elif dataset == 'whole':
        students = [rng.randint(0, (len(info_data)-1)) for _ in range(batch_size)] 
    else:
        students = [rng.randint(split_index+1, len(info_data)-1) for _ in range(batch_size)] 
    imgs = [rng.randint(0, len(info_data[i])-1) for i in students]
    strips_loc = [rng.randint(0, len(info_data[i][0][1])-1) for i in students]
    id_pairs = []
    for i in range(batch_size):
        std = students[i]
        img = imgs[i]
        exl = [(img+j) % len(info_data[std]) for j in range(examples)]
        srl = strips_loc[i]
        strips1 = [ info_data[std][e][1][srl] for e in exl ]
        strips2 = []
        if i >= batch_size // 2:
            img2 = (img + examples + rng.randint(1, len(info_data[std])-1)) % len(info_data[std])
            exl2 = [(img2+j) % len(info_data[std]) for j in range(examples)]
            strips2 = [ info_data[std][e][1][srl] for e in exl2]
        else:
            std2 = (std + rng.randint(1, len(info_data)-1)) % len(info_data)
            img2 = rng.randint(0, len(info_data[std2])-1)
            srl2 = rng.randint(0, len(info_data[std2][0][1])-1)
            exl2 = [(img2+j) % len(info_data[std2]) for j in range(examples)]
            strips2 = [ info_data[std2][e][1][srl2] for e in exl2]
        for k in range(examples):
            pairs[0][i,k,:,:,:] = cv2.imread(strips1[k])/255
            pairs[1][i,k,:,:,:] = cv2.imread(strips2[k])/255
        id_pairs.append((strips1, strips2))
    if with_id:
        return pairs, labels, id_pairs
    else:
        return pairs, labels

In [ ]:
p,l,ids = get_batch(info_data, 10, 4, True)

In [ ]:
for x in zip(l, ids):
    print(x[0])
    for a in x[1]:
        print(a)

In [5]:
def generate(info_data, examples, batch_size, dataset='train'):
    while True:
        pairs, labels = get_batch(info_data, batch_size, examples, False, dataset=dataset)
        yield(pairs, labels)

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K

In [18]:
def sequential_block(input_shape = (256,256,3), base_filters=64):
    model = keras.Sequential()
    model.add(layers.Conv2D(base_filters, (3,3), input_shape=input_shape))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(layers.BatchNormalization(renorm=True))
    model.add(layers.Conv2D(base_filters*4, (3,3)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(layers.BatchNormalization(renorm=True))
    model.add(layers.Conv2D(base_filters*4, (5,5)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(layers.BatchNormalization(renorm=True))
    model.add(layers.Conv2D(base_filters*4, (5,5)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(layers.BatchNormalization(renorm=True))
    model.add(layers.Conv2D(base_filters*4, (7,7)))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=2))
    model.add(layers.BatchNormalization(renorm=True))
    model.add(layers.Flatten())
    model.add(layers.Dense(base_filters*64, activation='relu'))
    
    
    
    return model

In [19]:
seq = sequential_block(base_filters=32)

In [20]:
seq.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 127, 127, 32)      0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 127, 127, 32)      224       
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 125, 125, 128)     36992     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 62, 62, 128)       0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 62, 62, 128)       896       
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 58, 58, 128)      

In [21]:
def get_siamese_model(input_shape = (4, 256,256,3), base_filters=16):

    left_input = layers.Input(input_shape)
    right_input = layers.Input(input_shape)
    
    # Convolutional Neural Network
    subshape = input_shape[1:4]
    model = sequential_block(subshape, base_filters)
    
    # Generate the encodings (feature vectors) for the two images
    encodeds_l = [model(left_input[:,k,...]) for k in range(input_shape[0])]
    encodeds_r = [model(right_input[:,k,...]) for k in range(input_shape[0])]

    encoded_l = layers.Average()(encodeds_l)
    encoded_r = layers.Average()(encodeds_r)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = layers.Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = layers.Dense(1,activation='sigmoid')(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = keras.Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [22]:
model = get_siamese_model(base_filters=40)
model.summary(200)

Model: "model"
________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
input_1 (InputLayer)                                              [(None, 4, 256, 256, 3)]                    0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
input_2 (InputLayer)                                              [(None, 4, 256, 256, 3)]                    0                                                                      

In [23]:
initial_learning_rate = 1e-5
end_learning_rate = 1e-7
decay_steps = 12800
lr_schedule = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate, decay_steps, end_learning_rate, power=0.5
)
model.compile(
        loss="binary_crossentropy",
        optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
        metrics=[keras.metrics.Precision(name='precision'),
             keras.metrics.Recall(name='recall'), "accuracy"] 
    )
run_name = "last_hack01-renorm"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs/"+run_name)

In [24]:
batch_size = 8
epochs = 100
examples = 4
steps_epoch = 128

In [ ]:
model.fit(generate(info_data, examples, batch_size), 
         epochs = epochs,
         steps_per_epoch= steps_epoch,
          callbacks=[tensorboard_callback],
          validation_batch_size=8,
          validation_data=val_set,
         shuffle = True)

Epoch 1/100
128/128 [==============================] - 147s 1s/step - loss: 0.7415 - precision: 0.5349 - recall: 0.5947 - accuracy: 0.5347 - val_loss: 0.7066 - val_precision: 0.5200 - val_recall: 0.3900 - val_accuracy: 0.5150
Epoch 2/100
128/128 [==============================] - 131s 1s/step - loss: 0.7181 - precision: 0.5194 - recall: 0.3248 - accuracy: 0.5113 - val_loss: 0.7406 - val_precision: 0.5778 - val_recall: 0.2600 - val_accuracy: 0.5350
Epoch 3/100
128/128 [==============================] - 131s 1s/step - loss: 0.6798 - precision: 0.5896 - recall: 0.3796 - accuracy: 0.5564 - val_loss: 0.7466 - val_precision: 0.5256 - val_recall: 0.4100 - val_accuracy: 0.5200
Epoch 4/100
128/128 [==============================] - 131s 1s/step - loss: 0.6789 - precision: 0.5603 - recall: 0.3776 - accuracy: 0.5404 - val_loss: 0.7268 - val_precision: 0.6400 - val_recall: 0.3200 - val_accuracy: 0.5700
Epoch 5/100
128/128 [==============================] - 131s 1s/step - loss: 0.6778 - precision: 

In [ ]:
model.save("last-hack02.h5")

In [ ]:
model.load_weights("last-hack02.h5")

### Validation:
Simple, just feed pairs from the validation split and see the performance.

In [1]:
val_set = get_batch(info_data, 500, 4, False, 'validation')

NameError: name 'get_batch' is not defined

In [ ]:
preds = model.predict(val_set[0])

In [ ]:
corr = [y == (p>0) for y, p in zip (val_set_y, preds)]

In [ ]:
round(preds[0][0])

In [30]:
cs = np.zeros((2,2))
for y,p in zip (val_set[1], preds):
    cs[int(y), round(p[0])] += 1 
acc = (cs[0,0] + cs [1 , 1]) / np.sum(cs)
rec = cs[1,1] / np.sum(cs[1])
print(cs, acc, rec)

[[84. 16.]
 [74. 26.]] 0.55 0.26


### Extended validation:
Take two images, and compare strip by strip, then do a majority voting.

In [ ]:
def get_val_batch(info_data, num_whole_images, batch_size):
    images = []
    split_index = int((len(info_data)-1) * 0.8)
    students = [rng.randint(split_index+1, len(info_data)-1) for _ in range(num_whole_images)] 
    labels = np.zeros((num_whole_images, ))
    labels[num_whole_images//2:] = 1
    id_pairs_img = []
    for j in range(num_whole_images):
        pairs = [np.zeros((batch_size, STRIP_SIZE, STRIP_SIZE, 3)) for i in range(2)]
        imgs = [rng.randint(0, len(info_data[i])-1) for i in students]
        std = students[j]
        img = imgs[j]
        id_pairs = []
        std2 = (std + rng.randint(1, len(info_data)-1)) % len(info_data)
        if j >= num_whole_images // 2:   
            img2 = (img + rng.randint(1, len(info_data[std])-1)) % len(info_data[std])
            std2 = std
        else:
            std2 = (std + rng.randint(1, len(info_data)-1)) % len(info_data)
            img2 = rng.randint(0, len(info_data[std2])-1)
        if batch_size > len(info_data[std][img][1]):
            print("a", info_data[std][0][0])
            std = std+1
            
        if batch_size > len(info_data[std2][img2][1]):
            print("b", info_data[std2][0][0])
            std2 = std+1
            
        for i in range(batch_size):    
            strip1 = info_data[std][img][1][i]
            strip2 = info_data[std2][img2][1][i]
#             print(strip1, strip2)
            pairs[0][i,:,:,:] = cv2.imread(strip1)/255
            pairs[1][i,:,:,:] = cv2.imread(strip2)/255
            id_pairs.append((strip1, strip2))
        images.append(pairs)
        id_pairs_img.append(id_pairs)
    
    return images, labels, id_pairs_img

In [ ]:
t_i, t_l, t_d = get_val_batch(info_data, 100, 10)

In [ ]:
cs = np.zeros((2,2))
cs_all = np.zeros((2,2))
for i, l, d in zip(t_i, t_l, t_d):
    preds = model.predict(i)
    for p in preds:
        cs_all[int(l), round(p[0])] += 1
#     print(int(l), round(np.mean(preds)), (np.mean(preds)))
    fp = sum([p[0] > 0.5 for p in preds]) > len(preds)//2 
    cs[int(l), int(fp)] +=1
#     print(l, d[0])

acc = (cs[0,0] + cs [1 , 1]) / np.sum(cs)
rec = cs[1,1] / np.sum(cs[1])
print(cs, acc, rec)
cs = cs_all
acc = (cs[0,0] + cs [1 , 1]) / np.sum(cs)
rec = cs[1,1] / np.sum(cs[1])
print(cs, acc, rec)